# Lecture 02: Production and decay of neutral variation

### Genetic variation in the infinite alleles model

We saw that genetic drift removes genetic variation over time as alleles fix or are lost. Mutation helps to counteract this progressive loss by introducing new genetic variation into populations.

One model for this process is the [infinite allele model](https://en.wikipedia.org/wiki/Infinite_alleles_model), where we assume that there is some rate $2u$ for mutations per reproduction cycle (so $2Nu$ over a population of $N$ individuals), and that each mutation results in a new allele. Kimura and Crow used this model to predict homozygosity and the number of alleles maintained in a population. Specifically, the probability $G$ that an individual is homozygous is predicted to be 

$$ G = \frac{1}{1 + 4 N u}\,, $$

and (the lower bound for) the number of alleles $n$ maintained in the population is 

$$ n \sim \frac{1}{G} = 1 + 4 N u\,. $$

Below, we'll write some code to simulate this model and to check its predictions using different values for the population size and mutation rate.

In [ ]:
# Import libraries for the simulation

import numpy as np
import numpy.random as rng

# First, let's write down the parameters of the model
# As before, choose your own values!

N = 100  # population size
u = 0.05  # rate of (neutral) mutations
n_g = 1000  # number of generations to simulate


# Next, let's make some lists to store data from the simulation
    
homozygosity = np.zeros(n_g)  # homozygosity of the population at each time
num_alleles = np.zeros(n_g)  # total number of alleles in the population at each time

In [ ]:
# Now, let's write the simulation itself
# There are many ways to do this, but we could start by making a list where each entry in the
# list corresponds to a different allele, and the number of individuals with that allele is the
# element in the list. We'll write a loop, where during each loop we'll propagate the population,
# count the number of mutations, and add that many new alleles to the population (while removing
# some of the old ones to compensate). 

# Option 1 -- start with no variation
allele_n = [2*N]  # each entry in the list is an allele, and the number
                  # stored there is the number of that allele in the population
    
# Option 2 -- start with a certain number of alleles, with even numbers of each
num = 10  # number of starting alleles
allele_n = [2*N/num]*num


# Before getting to the simulation, let's also help ourselves out by writing a function to
# compute homozygosity from a list of allele numbers

def get_h(alleles):
    p = np.array(alleles)/np.sum(alleles)
    h = 0
    for i in range(len(p)):
        h += p[i]**2
    return h


# Finally, let's continue on to the simulation

for i in range(n_g):
    
    # First, store the homozygosity and number of alleles
    homozygosity[i] = get_h(allele_n)
    num_alleles[i] = len(allele_n)
    
    # Next, check to see how many new mutations there are
    new_alleles = rng.binomial(n=2*N, p=u)
    
    # Next, update the number of alleles after reproduction
    new_allele_n = list(rng.multinomial(n=2*N-new_alleles, pvals=np.array(allele_n)/np.sum(allele_n)))
    
    # Add new alleles to the end of the list
    for j in range(new_alleles):
        new_allele_n.append(1)
        
    # Get rid of alleles that have disappeared (allele_n[i]==0)
    new_allele_n = np.array(new_allele_n)[np.array(new_allele_n)>0]
    
    # Now update the population
    allele_n = new_allele_n

Now that we've simulated the infinite alleles model, let's plot the results and see how they compare to our expectations from theory (i.e., $G=1/(1+4 N u)$ and $n\geq 1 + 4Nu$).

In [ ]:
# Import libraries for plotting

import seaborn as sns            # import seaborn
import matplotlib.pyplot as plt  # and matplotlib

# First, let's plot homozygosity vs. expectations from theory

G_theory = 1 / (1 + (4*N*u))
plt.hlines(y=G_theory, xmin=0, xmax=n_g, label='theory', color='black', ls='--')

sns.lineplot(x=np.arange(n_g), y=homozygosity, label='simulation')
plt.xlabel('Time (generations)')
plt.ylabel('Homozygosity, G')
plt.ylim(0, 1.1*np.max(homozygosity));

In [ ]:
# Next, let's plot the number of alleles vs. (lower bound) expectations from theory

n_theory = 1 + (4*N*u)
plt.hlines(y=n_theory, xmin=0, xmax=n_g, label='theory', color='black', ls='--')

sns.lineplot(x=np.arange(n_g), y=num_alleles, label='simulation')
plt.xlabel('Time (generations)')
plt.ylabel('Number of alleles, n')
plt.ylim(0, 1.1*np.max(num_alleles));